In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from collections import Counter
import sqlite3

In [2]:
#Code to import datafile from SQLITE file"
#con = sqlite3.connect(\"data/books_test_db.sqlite\")
#df = pd.read_sql_query(\"SELECT * from surveys\", con)
#df.head()

In [3]:
# Load the data
file_path = 'books.csv'
df = pd.read_csv(file_path, error_bad_lines = False)
df.head()

C:\Users\nmlam\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [4]:
# Convert the target column values to 1 and 0 based on their values
# if average rating is less than 3.5 = 0
# if average rating is more than 3.5 = 1
df['recommend'] = np.where((df['average_rating'] >= 3.50), 1, 0) 

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

df.reset_index(inplace=True, drop=True)

df.head(50)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,recommend
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,1
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,1
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,1
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,1
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,1
5,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.74,0976540606,9780976540601,en-US,152,19,1,4/26/2005,Nimble Books,1
6,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,0439827604,9780439827607,eng,3342,28242,808,9/12/2005,Scholastic,1
7,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,0517226952,9780517226957,eng,815,3628,254,11/1/2005,Gramercy Books,1
8,13,The Ultimate Hitchhiker's Guide to the Galaxy ...,Douglas Adams,4.38,0345453743,9780345453747,eng,815,249558,4080,4/30/2002,Del Rey Books,1
9,14,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams,4.22,1400052920,9781400052929,eng,215,4930,460,8/3/2004,Crown,1


In [5]:
df.dtypes

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                  int64
language_code          object
  num_pages             int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
recommend               int32
dtype: object

split and train data

In [6]:
# Create our features
X = pd.get_dummies(df, columns=['bookID', "average_rating", "language_code", "text_reviews_count",
                                "ratings_count", "recommend"])
#print(pd.get_dummies(X))

In [7]:
X = df.drop(columns=['recommend','title','authors','isbn','isbn13','publication_date','publisher', 'language_code'])
y = df["recommend"]

In [8]:
X.describe()

,bookID,average_rating,num_pages,ratings_count,text_reviews_count
count,11123.000000,11123.000000,11123.000000,1.112300e+04,11123.000000
mean,21310.856963,3.934075,336.405556,1.794285e+04,542.048099
std,13094.727252,0.350485,241.152626,1.124992e+05,2576.619589
min,1.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,10277.500000,3.770000,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,299.000000,7.450000e+02,47.000000
75%,32104.500000,4.140000,416.000000,5.000500e+03,238.000000
max,45641.000000,5.000000,6576.000000,4.597666e+06,94265.000000


In [9]:
y.value_counts()

1    10390
0      733
Name: recommend, dtype: int64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({0: 567, 1: 7775})

from challenge 17: oversampling
View the count of the target classes using Counter from the collections library.
Use the resampled data to train a logistic regression model.
Calculate the balanced accuracy score from sklearn.metrics.
Print the confusion matrix from sklearn.metrics.
Generate a classication report using the imbalanced_classification_report from imbalanced-learn.

In [11]:
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler
## instantiate the model
ros = RandomOverSampler(random_state=1)
## resample the targets
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 7775, 1: 7775})

In [12]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)

## fit
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [13]:
## calculate predictions
y_pred = model.predict(X_test)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5965168513441913

In [15]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
## calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[  94   72]
 [ 976 1639]]


In [16]:
cm_df = pd.DataFrame(
    cm, index=['Actual 1', 'Actual 0'], columns=['Predicted 1', 'Predicted 0'])
cm_df

####---##### need to double check that the results match the confusion matrix output.

,Predicted 1,Predicted 0
Actual 1,94,72
Actual 0,976,1639


In [17]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.09      0.57      0.63      0.15      0.60      0.35       166
          1       0.96      0.63      0.57      0.76      0.60      0.36      2615

avg / total       0.91      0.62      0.57      0.72      0.60      0.36      2781

